In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
%matplotlib inline

## Loading the dataset 

In [ ]:
df = pd.read_csv("/kaggle/input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv")
df.head()

In [ ]:

df.describe()

In [ ]:
df.info()

## Preprocessing of Dataset

In [ ]:
# find the null  values

df.isnull().sum()

In [ ]:
# fill the missing values for numerical terms - mean
df['LoanAmount']=df['LoanAmount'].fillna(df['LoanAmount'].mean())
df['Loan_Amount_Term']=df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mean())
df['Credit_History']=df['Credit_History'].fillna(df['Credit_History'].mean())

In [ ]:
# fill the missing values for categorical terms - mode
df['Gender']=df['Gender'].fillna(df['Gender'].mode()[0])
df['Married']=df['Married'].fillna(df['Married'].mode()[0])
df['Dependents']=df['Dependents'].fillna(df['Dependents'].mode()[0])
df['Self_Employed']=df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])

In [ ]:
df.isnull().sum()

## Exploratory Data Analysis

### Categorical Attributes Visualization

In [ ]:
sns.countplot(df['Gender'])

In [ ]:
sns.countplot(df['Dependents'])

In [ ]:
sns.countplot(df['Education'])

In [ ]:
sns.countplot(df['Self_Employed'])

In [ ]:
sns.countplot(df['Property_Area'])

In [ ]:
sns.countplot(df['Loan_Status'])

### Numerical attributes visualization

In [ ]:
sns.distplot(df['ApplicantIncome'])

In [ ]:
# apply Log Transformation to the attribute
df['ApplicantIncome']=np.log(df['ApplicantIncome'])

In [ ]:
sns.distplot(df['ApplicantIncome'])

In [ ]:
sns.distplot(df['CoapplicantIncome'])

In [ ]:

df['CoapplicantIncome']=np.log(df['CoapplicantIncome']+1)

In [ ]:
sns.distplot(df['CoapplicantIncome'])

In [ ]:
sns.distplot(df['LoanAmount'])

In [ ]:
df['LoanAmount']=np.log(df['LoanAmount']+1)

In [ ]:
sns.distplot(df['LoanAmount'])

In [ ]:
sns.distplot(df['Loan_Amount_Term'])

In [ ]:
df['Loan_Amount_Term']=np.log(df['Loan_Amount_Term'])

In [ ]:
sns.distplot(df['Loan_Amount_Term'])

In [ ]:
sns.distplot(df['Credit_History'])

### Creation of new attributes

In [ ]:
# total income
df['Total_Income']=df['ApplicantIncome'] + df['CoapplicantIncome']
df.head()

## Coorelation Matrix

In [ ]:
corr = df.corr()
sns.heatmap(corr,annot=True, cmap='BuPu')

In [ ]:
df.head()

In [ ]:
# drop unnecessary columns
cols = ['Loan_ID', 'CoapplicantIncome']
df=df.drop(columns=cols, axis=1)
df.head()

## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols=['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status', 'Dependents']
le=LabelEncoder()
for col in cols:
    df[col]=le.fit_transform(df[col])

In [ ]:
df.head()

## Train-Test Split

In [ ]:
# Specify input and output attributes
X=df.drop(columns=['Loan_Status'],axis=1)
y=df['Loan_Status']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

## Model Training

In [ ]:
# classify function
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
def classify(model,x,y):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)
    model.fit(x_train,y_train)
    print("Accuracy is ", model.score(x_test,y_test)*100)
    score=cross_val_score(model,x,y,cv=5)
    print("Cross validation is ," ,np.mean(score)*100)
    y_pred=model.predict(x_test)
    confusionmatrix(y_pred,y_test)
def confusionmatrix(y_pred,y_test):
    cm=confusion_matrix(y_test,y_pred)
    print(cm)
    sns.heatmap(cm,annot=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model,X,y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
classify(model,X,y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
model = RandomForestClassifier()
classify(model,X,y)

In [ ]:
model = ExtraTreesClassifier()
classify(model,X,y)

## Hyperparameter Tuning

In [ ]:
model = RandomForestClassifier(n_estimators=100,min_samples_split=25, max_depth=7,max_features=1)
classify(model, X,y)